In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json
from firebase_admin import auth
import names
from random import randint

In [2]:

cred = credentials.Certificate("/Users/waichung/Desktop/SAAD/testing/Submission Code for testing/leagueofselegends-firebase-adminsdk-roqv9-fb3a0feb19.json")
firebase_admin.initialize_app(cred)

## Populating database with questions

In [3]:

db = firestore.client()

In [4]:
def getData(jsonPath="/Users/waichung/Desktop/SAAD/testing/Submission Code for testing/Questionaire_Testing.json"):
    file= open(jsonPath)
    data=json.load(file)
    return data

In [5]:
ansMap={
    "a":1,
    "b":2,
    "c":3,
    "d":4,
}

In [19]:
def addQuestionToFirebaseByID(qnIdDict,world="Testing",section="Functional Testing",difficulty="Easy"):
    """Function to add question from json file into firestore given qnIdDict, world, section, and difficulty"""
    col_ref=db.collection(f"QnA/{world}/Sections/{section}/difficulty/{difficulty}/Questions")
    data=getData()
    new_data={}
    
    document=data[difficulty][qnIdDict]
    new_data["Question"]=document["question"]
    new_data["CorrectAnswer"]=ansMap[document["answers"].pop("correct")]
    optionList=[]
    for key,value in document["answers"].items():
        optionList.append(value)
    new_data["Answers"]=optionList

   

    doc_ref = db.document(f"QnA/{world}/Sections/{section}/difficulty/{difficulty}")
    doc = doc_ref.get()
    counter = doc.to_dict()["counter"]
    newCount=counter+1
    new_data["questionUID"] = f"Q{newCount}"

    #writing data into firebase
    col_ref.document(f"Q{newCount}").set(new_data)
    print(f"Successfully added {difficulty} Qn{qnIdDict}!")

    doc_ref.update({u'counter': newCount})

In [20]:
def addQuestionToFirebaseByDifficulty(difficulty,world="Testing",section="Functional Testing"):
    data=getData()
    document=data[difficulty]
    for key in document.keys():
        addQuestionToFirebaseByID(key,world,section,difficulty)

In [21]:
addQuestionToFirebaseByDifficulty("Easy")
addQuestionToFirebaseByDifficulty("Medium")
addQuestionToFirebaseByDifficulty("Hard")

Successfully added Easy Qn1!
Successfully added Easy Qn2!
Successfully added Easy Qn3!
Successfully added Easy Qn4!
Successfully added Easy Qn5!
Successfully added Easy Qn6!
Successfully added Easy Qn7!
Successfully added Easy Qn8!
Successfully added Easy Qn9!
Successfully added Medium Qn1!
Successfully added Medium Qn2!
Successfully added Medium Qn3!
Successfully added Medium Qn4!
Successfully added Medium Qn5!
Successfully added Medium Qn6!
Successfully added Medium Qn7!
Successfully added Medium Qn8!
Successfully added Medium Qn9!
Successfully added Medium Qn10!
Successfully added Hard Qn1!
Successfully added Hard Qn2!
Successfully added Hard Qn3!
Successfully added Hard Qn4!
Successfully added Hard Qn5!
Successfully added Hard Qn6!
Successfully added Hard Qn7!
Successfully added Hard Qn8!
Successfully added Hard Qn9!
Successfully added Hard Qn10!


## Populating Database with user info

1. Retrieving all uuid & email of users from firebase

In [70]:
uidList=[]
emailList=[]
page = auth.list_users()
while page:
    # for user in page.users:
    #     print('User: ' + user.uid)
    # Get next batch of users.
    page = page.get_next_page()

# Iterate through all users. This will still retrieve users in batches,
# buffering no more than 1000 users in memory at a time.
for user in auth.list_users().iterate_all():
    uidList.append(user.uid)
    emailList.append(user.email)


2. Deleting users from firebase authetication using UUid

In [71]:
from firebase_admin import auth

result = auth.delete_users(uidList)

print('Successfully deleted {0} users'.format(result.success_count))
print('Failed to delete {0} users'.format(result.failure_count))
for err in result.errors:
    print('error #{0}, reason: {1}'.format(result.index, result.reason))

Successfully deleted 50 users
Failed to delete 0 users


3. Deleting users from firebase firestore

In [66]:
for email in emailList:
    try:
        db.collection(u'Users').document(email).delete()
    except Exception as e:
        print(e)


### Creating user account for both firebase authentication and firestore

In [72]:
def createUser(emailAddress,password,matricNo,userName,ChallengeModeWins,StoryModeScore):
    
    user = auth.create_user(
    email=emailAddress,
    password=password,
    )
    
    
    data = {
        u'EmailAddress': emailAddress,
        u'MatriculationNo': matricNo,
        u'UserName': userName,
        u'UserProgressLevel':u"Easy",
        u'ChallengeModeWins':ChallengeModeWins,
        u'StoryModeScore':StoryModeScore,

    }
    db.collection(u'Users').document(emailAddress).set(data)

    print(f'Sucessfully created new user: {user.email}')

In [73]:


def populateDatabase(numOfAcc):
    nameList=[]
    while len(nameList)<numOfAcc:
        name= names.get_first_name()
        if name not in nameList:
            nameList.append(name)

    
    for name in nameList:
        try:
            emailAddress= name+"@gmail.com"
            password="StrongPassword"
            matricNo="U"+str(randint(1000000, 9999999))+"H"
            userName=name
            challengeModeWins=randint(0, 20)
            StoryModeScore=0
            createUser(emailAddress, password,matricNo,userName,challengeModeWins,StoryModeScore)
        except Exception as e:
            print(e)



In [74]:
populateDatabase(50)

Sucessfully created new user: tim@gmail.com
Sucessfully created new user: beverly@gmail.com
Sucessfully created new user: gregory@gmail.com
Sucessfully created new user: irene@gmail.com
Sucessfully created new user: joyce@gmail.com
Sucessfully created new user: david@gmail.com
Sucessfully created new user: tammy@gmail.com
Sucessfully created new user: franklin@gmail.com
Sucessfully created new user: peter@gmail.com
Sucessfully created new user: emerson@gmail.com
Sucessfully created new user: mamie@gmail.com
Sucessfully created new user: craig@gmail.com
Sucessfully created new user: juanita@gmail.com
Sucessfully created new user: christopher@gmail.com
Sucessfully created new user: george@gmail.com
Sucessfully created new user: shane@gmail.com
Sucessfully created new user: leslee@gmail.com
Sucessfully created new user: kevin@gmail.com
Sucessfully created new user: ronald@gmail.com
Sucessfully created new user: robert@gmail.com
Sucessfully created new user: ray@gmail.com
Sucessfully creat